In [1]:
import time
start_time = time.time()

import sys
sys.path.append(r"D:\MINJI\CLASS\2024-2\(TSR)Structural Reliability\03. Term Project\code\final")

import itertools
import numpy as np
import networkx as nx
from scipy.stats import multivariate_normal, norm
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from itertools import combinations
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import cpm, variable, operation
from BNS_JT import brc
import pandas as pd

# 모든 행, 열 출력
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
# Define the network
nodes = {
    "n1": (0, 1.5), "n2": (0, 0), "n3": (4, 3), "n4": (4, 1.5),
    "n5": (4, 0), "n6": (5.5, 0), "n7": (7.6, 3), "n8": (7.6, 1.5),
    "n9": (8, 0.7), "n10": (8, 0), "n11": (12, 3), "n12": (12, 1.5),
    "n13": (12, 0.9), "n14": (12, 0), "n15": (14, 1.5), "n16": (16, 3),
    "n17": (20, 2), "n18": (20, 0.5)
}

edges = {
    "e1": ("n1", "n2"), "e2": ("n1", "n3"), "e3": ("n1", "n4"),
    "e4": ("n2", "n5"), "e5": ("n3", "n4"), "e6": ("n4", "n5"),
    "e7": ("n3", "n7"), "e8": ("n4", "n8"), "e9": ("n5", "n6"), 
    "e10": ("n6", "n9"), "e11": ("n6", "n10"), "e12": ("n7", "n8"), 
    "e13": ("n8", "n9"), "e14": ("n9", "n10"), "e15": ("n7", "n11"),
    "e16": ("n8", "n12"), "e17": ("n9", "n13"), "e18": ("n10", "n14"),
    "e19": ("n11", "n12"), "e20": ("n12", "n13"), "e21": ("n13", "n14"),
    "e22": ("n11", "n16"), "e23": ("n12", "n15"), "e24": ("n13", "n15"),
    "e25": ("n14", "n18"), "e26": ("n15", "n16"), "e27": ("n15", "n18"),
    "e28": ("n16", "n17"), "e29": ("n17", "n18")
}

probs = {
    'e1': {0: 0.105722, 1: 0.894278},
    'e2': {0: 0.100468, 1: 0.899532},
    'e3': {0: 0.102776, 1: 0.897224},
    'e4': {0: 0.108221, 1: 0.891779},
    'e5': {0: 0.100162, 1: 0.899838},
    'e6': {0: 0.105256, 1: 0.894744},
    'e7': {0: 0.097419, 1: 0.902581},
    'e8': {0: 0.101082, 1: 0.898918},
    'e9': {0: 0.107908, 1: 0.892092},
    'e10': {0: 0.105256, 1: 0.894744},
    'e11': {0: 0.107126, 1: 0.892874},
    'e12': {0: 0.09833, 1: 0.90167},
    'e13': {0: 0.101235, 1: 0.898765},
    'e14': {0: 0.104634, 1: 0.895366},
    'e15': {0: 0.096209, 1: 0.903791},
    'e16': {0: 0.098787, 1: 0.901213},
    'e17': {0: 0.101389, 1: 0.898611},
    'e18': {0: 0.098482, 1: 0.901518},
    'e19': {0: 0.096662, 1: 0.903338},
    'e20': {0: 0.098939, 1: 0.901061},
    'e21': {0: 0.103394, 1: 0.896606},
    'e22': {0: 0.093956, 1: 0.906044},
    'e23': {0: 0.097419, 1: 0.902581},
    'e24': {0: 0.098482, 1: 0.901518},
    'e25': {0: 0.102159, 1: 0.897841},
    'e26': {0: 0.094705, 1: 0.905295},
    'e27': {0: 0.097267, 1: 0.902733},
    'e28': {0: 0.093658, 1: 0.906342},
    'e29': {0: 0.096209, 1: 0.903791}
}

varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

In [3]:
def net_conn(comps_st, od_pair, edges, varis): # maximum flow analysis

    G = nx.Graph()
    for k,x in comps_st.items():
        G.add_edge(edges[k][0], edges[k][1]) # we add each edge
        G[edges[k][0]][edges[k][1]]['capacity'] = varis[k].values[x] # define capacity as 0 if state = 0 or 1 if state = 1

    # perform maximum flow analysis between the OD pair
    G.add_edge(od_pair[1], 'new_d', capacity=1) # add a new edge with capacity 1 to ensure we find only ONE path. 
    f_val, f_dict = nx.maximum_flow(G, od_pair[0], 'new_d', capacity='capacity', flow_func=shortest_augmenting_path)

    if f_val > 0: # if the flow between the OD pair is greater than 0, the two nodes are connected
        sys_st = 's'

        # We can infer an associated minimum survival rule in case of network connectivity.
        min_comps_st = {} 
        for k, x in comps_st.items():
            k_flow = max([f_dict[edges[k][0]][edges[k][1]], f_dict[edges[k][1]][edges[k][0]]])
            if k_flow > 0: # the edges with flows > 0 consitute a minimum survival rule.
                min_comps_st[k] = 1

    else:
        sys_st = 'f'

        # In case of system failure, obtaining a minimum failure rule is not straightforward.
        min_comps_st = None

    return f_val, sys_st, min_comps_st

In [4]:
od_pair=('n2','n8')
sys_fun = lambda comps_st : net_conn(comps_st, od_pair, edges, varis)
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=np.inf, max_nb=np.inf, pf_bnd_wr=0.0)

[System function runs 200]..
The # of found non-dominated rules (f, s): 200 (94, 106)
Probability of branchs (f, s, u): (1.5440e-02, 9.85e-01, 8.8153e-06)
The # of branches (f, s, u), (min, avg) len of rf: 20271 (2670, 6442, 11159), (2, 5.89)
[System function runs 400]..
The # of found non-dominated rules (f, s): 400 (211, 189)
Probability of branchs (f, s, u): (1.5443e-02, 9.85e-01, 1.1868e-06)
The # of branches (f, s, u), (min, avg) len of rf: 39693 (13034, 11130, 15529), (2, 6.89)
[System function runs 600]..
The # of found non-dominated rules (f, s): 600 (332, 268)
Probability of branchs (f, s, u): (1.5443e-02, 9.85e-01, 1.7539e-07)
The # of branches (f, s, u), (min, avg) len of rf: 54792 (24534, 16532, 13726), (2, 7.52)
[System function runs 800]..
The # of found non-dominated rules (f, s): 800 (438, 362)
Probability of branchs (f, s, u): (1.5443e-02, 9.85e-01, 3.2957e-08)
The # of branches (f, s, u), (min, avg) len of rf: 61412 (31744, 19875, 9793), (2, 7.88)
[System function run

In [5]:
def calculate_beta(brs, probs):
    results = {}
    
    # Initialize variables
    components = list(probs.keys())  # List of all components
    state_combinations = []  # List to store N_c for each failure combination
    
    # Generate combinations of failure components
    for r in range(1, 7):  # r: number of failed components
        failure_combinations = list(itertools.combinations(components, r))
        state_combinations.extend(failure_combinations)

    results["Failure Combinations"] = state_combinations

    # Parse branches to identify B (failure branches)
    B = [branch for branch in brs]
    results["B"] = B

    comp_fail_prob = []

    for combination in state_combinations:
        branch_probability = 0
        expression_terms = []  # To store terms as strings

        # Filter branches where all components in the combination have down bound == 0
        filtered_B = [
            branch for branch in B
            if all(branch.down.get(component, 1) == 0 for component in combination)
        ]

        for branch in filtered_B:
            l_states = branch.down
            u_states = branch.up
            p_branch = branch.p

            # 1. Case: All components in combination satisfy u == 0
            if all(u_states.get(component, 1) == 0 for component in combination):
                branch_probability += p_branch
                expression_terms.append(f"{p_branch}")

            # 2. Case: Some components have u == 1 and l == 0
            else:
                # Identify components with u == 1 and l == 0
                relevant_components = [
                    component for component in combination
                    if u_states.get(component, 1) == 1 and l_states.get(component, 0) == 0
                ]

                if relevant_components:
                    # Compute failure probability product
                    failure_prob_product = np.prod([probs[comp][0] for comp in relevant_components])

                    # Calculate the term
                    term = p_branch * failure_prob_product
                    branch_probability += term
                    expression_terms.append(f"{p_branch} * prod({[probs[c][0] for c in relevant_components]})")

        comp_fail_prob.append(branch_probability)

    results["comp_fail_prob"] = comp_fail_prob


    # Calculate π_ij = -Φ^(-1)(P(F_sys | F_i, H_j))
    beta_results = []
    for i, prob in enumerate(comp_fail_prob):
        if prob > 0 and prob < 1:
            beta_ij = -norm.ppf(prob)
        else:
            beta_ij = float('inf')
        beta_results.append({
            'State': state_combinations[i], 
            'Failure Probability': prob,
            'Beta': beta_ij 
        })


    return beta_results, state_combinations, comp_fail_prob

In [ ]:
# 함수 호출
beta_results, state_combinations, failure_probabilities = calculate_beta(brs, probs)

# DataFrame 생성
df_beta_results = pd.DataFrame(beta_results)  # 열 이름은 beta_results의 키와 자동 매핑됩니다.
print(df_beta_results)
